In [20]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import numpy as np
import pandas as pd
import pickle
import os
import admix_prs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


# Load data

In [21]:
config = "hsq-0.25-pcausal-0.01-hermodel-uniform"

In [22]:
if os.path.exists("df_res_list.pkl"):
    with open("df_res_list.pkl", "rb") as f:
        df_res_list = pickle.load(f)
else:
    df_res_list = admix_prs.load_sim_data(config)
    with open("df_res_list.pkl", "wb") as f:
        pickle.dump(df_res_list, f)

In [25]:
len(df_res_list)

5

In [23]:
df = df_res_list[0].copy()
df["PHENO_G"] -= df["PHENO"].mean()
df["PHENO"] -= df["PHENO"].mean()

In [29]:
df

,PRS_MEAN,PRS_STD,PHENO,PHENO_G
FID_IID,,,,
3627963_3627963,-0.959645,0.328504,-1.330712,-0.960817
1381646_1381646,0.458950,0.260210,0.867125,0.664614
1846099_1846099,0.770227,0.241172,1.389261,0.639234
3028248_3028248,-0.209268,0.267718,-0.627948,-0.194213
3690795_3690795,-0.335288,0.290984,-1.260668,-0.948487
...,...,...,...,...
4931041_4931041,-0.847915,0.260484,-0.177411,-0.650833
1118302_1118302,-0.192384,0.257625,0.484706,-0.002039
2616445_2616445,-0.045958,0.257951,0.581634,-0.043358


In [18]:
admix_prs.stratify_calculate_r2(df, x_col="PHENO_G", y_col="PRS_MEAN")

TypeError: stratify_calculate_r2() missing 1 required positional argument: 'stratify_col'

In [19]:
admix_prs.eval_calibration(df, x_col="PHENO_G", y_col="PRS_MEAN")

TypeError: eval_calibration() got an unexpected keyword argument 'y_col'

# TODO:
1. stratify_calculate_r2 by PC1, PC2, AGE, SEX
- R2 different by PC1 and PC2, but not by AGE, SEX
- do this for both PHENO_G and PHENO

2. eval_calibration using PHENO_G and `[MEAN - 1.645 * STD, MEAN + 1.645 * STD]` 
- expect 90% coverage
3. eval_calibration using PHENO and `[MEAN - 1.645 * PHENO-STD, MEAN + 1.645 * PHENO-STD]`

- PHENO-G ~ PRS-MEAN + STD(PRS)
- PHENO ~ PRS-MEAN + STD(PRS + ENV)

STD(PRS + ENV) = sqrt(PRS-STD ** 2 + ENV-STD ** 2)

df["PHENO-STD"] = np.sqrt(df["PRS-STD"] ** 2 + 0.75)

# simulated heritability is 0.25
# var[y] = 1
# hsq = var[gv] / (var[gv] + var[e]) = 0.25
# var[e] = 0.75

ENV-STD ** 2 = 0.75 = 1 - 0.25